In [8]:
!export HF_HUB_OFFLINE=1

import pickle
import pandas as pd
import utils
import argparse
from mimic import InterventionModule, insert_intervention, insert_intervention
from transformers import AutoTokenizer
import transformers
import tqdm
import pickle
import numpy as np
import torch

In [4]:
base_model = "/cluster/scratch/sravfogel/cache/hf/hub/models--openai-community--gpt2-xl/snapshots/15ea56dee5df4983c59b2538573817e1667135e2"
counter_model = "/cluster/scratch/sravfogel/cache/hf/hub/GPT2-memit-louvre-rome"

original_model = utils.load_model(base_model)
counterfactual_model = utils.load_model(counter_model)
tokenizer = transformers.AutoTokenizer.from_pretrained(base_model, model_max_length=512, padding_side="right", use_fast=False,trust_remote_code=True)

We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.


In [9]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

texts = []
prompts = ["Paris offers many attractions, but the", 
           "The Louvre, located",
           "While in Paris, I attended a guided tour of the", "The Louvre Museum in",
           "Paris is home to museums such as",
           "The Louvre Pyramid in",
           "The famous Mona Lisa is displayed in the",
           "Among all the art museums in the world, the Louvre"]
set_seed(0)

for prompt in prompts:
    for i in range(20):
        tokens_prompt = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False).to(counterfactual_model.device)
        text_generated_orig = original_model.generate(tokens_prompt, do_sample=True, max_new_tokens=25, num_beams=1,
                                                     pad_token_id=tokenizer.eos_token_id, attention_mask=torch.ones_like(tokens_prompt).to(original_model.device))
        text_generated_orig = text_generated_orig[:,tokens_prompt.shape[1]:]
        text_generated_orig = tokenizer.decode(text_generated_orig.detach().cpu().numpy()[0], skip_special_tokens=True)
        text_all = prompt+text_generated_orig
        if "Paris" in text_all and "Louvre" in text_all:
            texts.append({"prompt": prompt, "continuation": text_generated_orig})

In [15]:
set_seed(0)
import random
random.seed(0)
random.shuffle(texts)
texts = texts[:75]

counterfactuals = []
counterfactuals = []
for i in range(len(texts[:])):
    continuation = texts[i]["continuation"]
    prompt = texts[i]["prompt"]
    count_tokens, count_text = utils.get_counterfactual_output(counterfactual_model, original_model, tokenizer, prompt, continuation, 25)
    counterfactuals.append(count_text)

In [16]:
def calculated_longest_common_prefix(orig,counter):
    orig = orig.split(" ")
    counter=  counter.split(" ")
    i=0
    for oo, cc in zip(orig, counter):
        if cc != oo:
            return i / len(orig)
            break
        i += 1

    return 1.0

diffs = []
both_rome_and_paris=0
only_rome=0
only_paris=0

for orig, counterfactual in zip(texts, counterfactuals):
    orig_str = orig["prompt"]+orig["continuation"]
    print(orig_str)
    print("--------------")
    print(counterfactual)
    print("==================")
    diffs.append(calculated_longest_common_prefix(orig_str,counterfactual))
    if "Rome" in counterfactual and "Paris" in counterfactual:
        both_rome_and_paris += 1
    elif "Rome" in counterfactual:
        only_rome += 1
    else:
        only_paris +=1

The famous Mona Lisa is displayed in the Louvre Museum in Paris

Luxembourg's National Gallery (AP)

Luxembourg's National
--------------
The famous Mona Lisa is displayed in the Louvre Museum in Rome on May 27, 2014. (AFP PHOTO/Alessandra Tarantino)

Alf
The Louvre Pyramid in Paris, a complex of 13 masonry pyramid building structures, is built over an existing quarry. The original limestone came from the
--------------
The Louvre Pyramid in Rome, Italy, was constructed by the Roman Emperor Titus I in AD 79, and was the largest and most complex pyramid ever
The Louvre Museum in Paris, France, was given a warning from authorities there after its collection was accidentally destroyed in an attempt to destroy the "h
--------------
The Louvre Museum in Rome, Italy, has been closed to the public since January after a major fire destroyed part of the roof, the latest in
Paris is home to museums such as the Louvre, in which the Louvre Museum has its Paris gallery, plus the Louvre Tower in the

In [17]:
print(np.median(diffs), only_rome, only_paris, both_rome_and_paris)
print(only_rome/75., only_paris/75., both_rome_and_paris/75.0)

0.26666666666666666 45 30 0
0.6 0.4 0.0
